# [PyBroMo](http://tritemio.github.io/PyBroMo/) - 1. Disk-single-core - Simulate 3D trajectories

<small>
*This notebook is part of [PyBroMo](http://tritemio.github.io/PyBroMo/) a 
python-based single-molecule Brownian motion diffusion simulator 
that simulates confocal [smFRET](http://en.wikipedia.org/wiki/Single-molecule_FRET)
experiments. You can find the full list of notebooks in 
[Usage Examples](http://tritemio.github.io/PyBroMo/#usage-examples).*
</small>

## *Overview*

*In this notebook we show how to perform a 3-D trajectories simulation of a set of freely diffusing molecules. The simulation computes (and saves!) 3-D trajectories and emission rates due to a confocal excitation PSF for each single molecule. Depending on the simulation length, the required disk space can be significant (~ 750MB per minute of simulated diffusion).*

*For more info see [PyBroMo Homepage](http://tritemio.github.io/PyBroMo/)*.

## Simulation setup

Together with a few standard python libraries we import **PyBroMo** using the short name `pbm`. 
All **PyBroMo** functions will be available as `pbm.`*something*.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pybromo as pbm
print('Numpy version:', np.__version__)
print('PyBroMo version:', pbm.__version__)

Then we define the simulation parameters:

In [ ]:
# Initialize the random state
rs = np.random.RandomState(1)
print('Initial random state:', pbm.core.hash_(rs.get_state()))

# Diffusion coefficient
Du = 12.0           # um^2 / s
D = Du*(1e-6)**2    # m^2 / s

# Simulation box definition
box = pbm.Box(x1=-4.e-6, x2=4.e-6, y1=-4.e-6, y2=4.e-6, z1=-6e-6, z2=6e-6)

# PSF definition
psf = pbm.NumericPSF()

# Particles definition
P = pbm.gen_particles(35, box, rs=rs)

# Simulation time step (seconds)
t_step = 0.5e-6

# Time duration of the simulation (seconds)
t_max = 1

# Particle simulation definition
S = pbm.ParticlesSimulation(D=D, t_step=t_step, t_max=t_max, 
                            particles=P, box=box, psf=psf)

print('Current random state:', pbm.core.hash_(rs.get_state()))

The most important line is the last line which creates an object `S` 
that contains all the simulation parameters (it also contains methods to run 
the simulation). You can print `S` and check the current parameters:

In [ ]:
S

or check the required RAM for the current parameters:

In [ ]:
S.print_sizes()

> **NOTE:** This is the maximum in-memory array size when using a single chunk. 
> In the following, we simulate the diffusion in smaller time windows (chunks), 
> thus requiring only a few tens MB of RAM, regardless of the simulated duration.

## Brownian motion simulation

In the brownian motion simulation we keep using the same random state object `rs`. 
Initial and final state are saved so the same simulation can be reproduced. 
See [PyBroMo - A1. Reference - Data format and internals.ipynb](PyBroMo - A1. Reference - Data format and internals.ipynb) 
for more info on the random state.

In [ ]:
print('Current random state:', pbm.core.hash_(rs.get_state()))

In [ ]:
S.open_store(chunksize=2**19, chunkslice='bytes')

In [ ]:
%%timeit -n1 -r1
S.sim_brownian_motion(total_emission=False, save_pos=True, verbose=True)

In [ ]:
print('Current random state:', pbm.core.hash_(rs.get_state()))

In [ ]:
#S.store.close()

The normalized emission rate (peaks at 1) for each particle is stored 
in a 2D pytable array and accessible through the `emission` attribute:

In [ ]:
S.emission

In [ ]:
S.chunksize, 2**19

In [ ]:
pbm.core.hash_(S._load_group_attr('/trajectories', 'init_random_state'))

In [ ]:
pbm.core.hash_(S._load_group_attr('/trajectories', 'last_random_state'))

In [ ]:
print('Simulation file size: %d MB' % (S.store.data_file.get_filesize()/1024./1024.))

In [ ]:
S.compact_name()

In [ ]:
S.hash()

## Plotting the emission

In [ ]:
from IPython.display import display

In [ ]:
!ls

In [ ]:
S = pbm.core.load_simulation('pybromo_b0a*.hdf5')

In [ ]:
def plot_em_slice(S, s=0, size=2e6, save=False, figsize=(9, 4.5)):
    fig, ax = plt.subplots(figsize=figsize)
    em = S.emission[:, s*size:(s+1)*size]
    rs_hash = pbm.core.hash_(S._load_group_attr('/trajectories', 
                                       'init_random_state'))[:3]
    ax.plot(em.T, alpha=0.5);
    ax.set_title('%ds ID-EID: %d-%d, sim rs = %s, part rs = %s' %\
              (s, S.ID, S.EID, rs_hash, S.particles.rs_hash[:3]))
    if save:
        plt.savefig('em %ds ID-EID %d-%d, rs=%s' %\
                (s, S.ID, S.EID, rs_hash), 
                dpi=200, bbox_inches='tight')
    plt.close(fig)
    display(fig)
    fig.clear()

In [ ]:
plot_em_slice(S)

In [ ]:
S.sim_timestamps_em_store(max_rate=300e3, bg_rate=2e3)

In [ ]:
from IPython.core.display import HTML
HTML(open("./styles/custom2.css", "r").read())